In [1]:
from scripts.functions import *
from parameters import *

<IPython.core.display.Javascript object>

In [2]:
def download_from_orders(current_orders, start_date, stop_date, order_name=None, order_id=None, folder_name=''):
    """Will download the images in the current_orders dataframe.
    
    Args:
        current_orders: (pd.DataFrame) DataFrame containing the current server orders
        start_date: (datetime.datetime) Filter the orders which start from the start_date
        stop_date: (datetime.datetime) Filter the orders before the stop_date
        order_name: (str) (optional): Partial or complete order name, will be used to filter the current_orders dataframe
        order_id: (str) (optional): Partial or complete id_name, will be used to filter the current_orders dataframe
        folder_name: (str) (optional): Sub folder created in the downloads folders
    
    Return:
        Download orders in the downloads/ folder.
    """

    now = datetime.datetime.now()
    formated_now = now.strftime('%Y%m%d_%H_%M')
    dw_log_file = os.path.join(LOG_PATH, f'download_logs_{formated_now}.txt')
    success_states = ['success', 'partial']
    
    if order_name:
        current_orders = current_orders[current_orders.name.str.contains(order_name)]
    
    elif order_id:
        current_orders = current_orders[current_orders.id.str.contains(order_id)]
        
    
    for idx, row in current_orders.iterrows():

        created_on = pd.to_datetime(row.created_on).date()
        state = row.state
        sample_name = row['name']
        order_id = row.id
        
        if state in success_states:
            
            if created_on >= start_date and created_on <= stop_date:

                download_order_path = os.path.join(DOWNLOAD_PATH, str(folder_name), str(sample_name))
                Path(download_order_path).mkdir(parents=True, exist_ok=True)

                # Check if there is a .zip file inside the folder
                if not any(['.zip' in f for f in os.listdir(download_order_path)]):
                    try:
                        print(f'downloading {sample_name} ')
                        callback = api.write_to_file(directory=f'{download_order_path}/', overwrite=True)

                        @backoff.on_exception(backoff.expo,(planet.api.exceptions.OverQuota,
                                                           planet.api.exceptions.TooManyRequests),max_time=360)
                        def download():
                            response = client.download_order(order_id, callback=callback)
                            response[0].wait().response
                            return response

                        download()
                        

                    except Exception as e:
                        print(f'There was an error with {sample_name}, please check download log file.')
                        with open(dw_log_file, 'a') as lf:
                            lf.write(f'Sample {sample_name}:{e}\n')


## Download from existing planet orders

## Start the client
To use your own api keys and parameters, copy paste the `parameters.py.dist` file in the same folder and remove the `.dist` extention. You can then replace the string with your own keys. only the .dist will be pushed to the dist git rep. 

In [3]:
client = api.ClientV1(api_key=PLANET_API_KEY)

# 1. Get all the current orders and store them in a dataframe
This process could take some minutes, after created, you can filter the dataframe

## Get status
The following get_order_status line has to be re-runned everytime we want to know the orders statusw

In [4]:
%%time
# Use pages=None to query over all the server orders.
current_orders = get_orders_status(client, pages=None).sort_values(by=['created_on'])
current_orders

CPU times: user 1.18 s, sys: 73.6 ms, total: 1.25 s
Wall time: 1min 58s


,created_on,last_modified,id,name,state


### Start download process

In [7]:
%%time
# Select the dates in which you want to download the images
start_date = datetime.date(2020,9,21)
stop_date = datetime.date(2020,9,21)

# Specify an order_name or order_id to filter the downloads.
# If not order_name and not order_name, it will download all the orders
# between the given dates, with the given dataframe.

# You can use partial order_name and order_id
# For example:
#            order_name='2f2a7561-9bbd-'
#            order_id='sample'


x = download_from_orders(current_orders, 
                     start_date, 
                     stop_date, 
                     order_name=None, 
                     order_id=None, 
                     folder_name='Indonesia_2109'
                    )

CPU times: user 1.54 s, sys: 297 ms, total: 1.84 s
Wall time: 6.04 s


In [6]:
from IPython.display import Audio
sound_file = 'http://soundbible.com/mp3/Home%20Phone%20Ringing-SoundBible.com-476855293.mp3'
Audio(sound_file, autoplay=True)